<a href="https://colab.research.google.com/github/Yussufkadir/Book-recommender/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import vstack, hstack, csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import regex as re
import unicodedata
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-01-26 18:49:41--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.09s   

2025-01-26 18:49:41 (264 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books.shape

(271379, 3)

In [ ]:
df_ratings.shape

(1149780, 3)

In [ ]:
user_ratings_count = df_ratings['user'].value_counts()
ratings = df_ratings[df_ratings['user'].isin(user_ratings_count[user_ratings_count >= 12].index)]

book_ratings_count = df_ratings['isbn'].value_counts()
ratings = df_ratings[df_ratings['isbn'].isin(book_ratings_count[book_ratings_count >= 12].index)]

ratings = pd.merge(ratings, df_books[['isbn', 'title']], on='isbn')

print(ratings.shape)

user_book_matrix = ratings.pivot_table(index='title', columns='user', values='rating').fillna(0)

user_book_matrix_sparse = csr_matrix(user_book_matrix.values)

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(user_book_matrix_sparse)

(482497, 4)


NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
def get_recommends(book_name=""):
    if book_name not in user_book_matrix.index:
        return f"Book '{book_name}' not found in the dataset."

    book_index = user_book_matrix.index.get_loc(book_name)

    distances, indices = model.kneighbors(user_book_matrix.iloc[book_index, :].values.reshape(1, -1), n_neighbors=15)

    recommended_books = []
    for i in range(1, 5):
        recommended_books.append([
            user_book_matrix.index[indices.flatten()[i]],
            distances.flatten()[i]
        ])

    return [book_name, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['After the Parade', 0.9059894], ['Blood Relations', 0.90677667], ['The Lovely Bones: A Novel', 0.90959823], ['I Know This Much Is True', 0.9097615]]]
You haven't passed yet. Keep trying!
